# BB84 Quantum Key Distribution

This notebook demonstrates the BB84 Quantum Key Distribution protocol from Chapter 5, Single-Qubit Quantum Gates.

## 0. Setup

The cells below install the packages and import the libraries needed by this demo.

In [ ]:
#@title Install Qiskit and other dependencies
%pip install qiskit qiskit-aer qiskit-ibm-runtime pylatexenc

In [3]:
# Imports

from math import cos, sin, pi as π
from IPython.display import display, Markdown
from qiskit_aer import AerSimulator
from qiskit.circuit import ClassicalRegister, QuantumCircuit, QuantumRegister
from qiskit.quantum_info import partial_trace, Statevector

## 1. Quantum Random Number Generator

The following cell defines a Quantum Random Number Generator (QRNG) circuit. This circuit generates a random bit by preparing a qubit in the state $|+\rangle$, then measuring it in the $Z$ basis.

*Note*: The sample circuits in this repository use the Qiskit AER simulator. 
As such, all quantum randomness is simulated, actually using a deterministic pseudo-random number generator.
However, these same circuits could in principle be run on a real quantum computer to generate true quantum randomness.

In [10]:
def qrng():
    """
    Generate a random bit using a quantum random number generator (QRNG).

    This function creates a quantum circuit with a single qubit, applies a Hadamard gate to put the qubit
    into a superposition state, and then measures the qubit. The result is a random bit (0 or 1) due to
    the inherent randomness of quantum measurement.

    Returns:
        int: A random bit (0 or 1) generated by the quantum circuit.
    """
    q = QuantumRegister(1)
    circuit = QuantumCircuit(q)
    circuit.h(q)
    circuit.measure_all()
    counts = AerSimulator().run(circuit).result().get_counts()
    return int(list(counts.keys())[0])


bits = [qrng() for _ in range(8)]
display(Markdown(f"Random bits: {bits}"))

Random bits: [0, 1, 1, 0, 0, 1, 1, 0]

In [7]:
def run(n_bits, e_bits, eavesdropper=False):
    alice_bases = []
    bob_bases = []
    for i in range(n_bits):
        q = [q0] = QuantumRegister(1)
        c_alice = ClassicalRegister(1)
        c_eve = ClassicalRegister(1)
        c_bob = ClassicalRegister(1)
        circuit = QuantumCircuit(q, c_alice, c_eve, c_bob)
        if qrng():
            circuit.x(q0)
        if qrng():
            circuit.h(q0)
            alice_bases.append('X')
        else:
            alice_bases.append('Z')
        circuit.measure(q, c_alice)
        # TODO: eavesdropper
        if qrng():
            circuit.h(q0)
            bob_bases.append('X')
        else:
            bob_bases.append('Z')
        circuit.measure(q, c_bob)

